In [24]:
import re
import pdfplumber

In [45]:
with pdfplumber.open("data/cv_sample_8.pdf") as pdf:
    page = pdf.pages[0]
    text = page.extract_text()

### Email

In [52]:
email_pa = re.compile(r"[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+")
email = email_pa.search(text).group(0)
email

'moise.kouassi+cv12@exemple.com'

### Name

In [ ]:
name_pa = re.compile(r"^[A-Z][A-Z]+\s+[A-Za-zÀ-ÖØ-öø-ÿ]+\s+[A-Za-zÀ-ÖØ-öø-ÿ]+")
name = name_pa.search(text).group(0)
name

'KOUASSI Moïse Armand'

### Phone

In [54]:
phone_pa = re.compile(r"(?:\+225\s?)?(?:\d{2}\s?){5}")
phone = phone_pa.search(text).group(0)
phone = phone.replace("\n", " ").strip()
phone

'+225 07 67 21 83 04'

### Birthdate

In [56]:
bd_pa = re.compile(r"(\d{2}/\d{2}/\d{4})")
bd = bd_pa.search(text).group(0)
bd

'12/09/1998'

In [ ]:
email = re.compile(r'[a-z]+\.[a-z]+\+["cv"{2}]')
profile = 
birth_date = 


In [35]:
for line in text.split("\n"):
    print(line)

Thomas Durand
Commercial — En alternance
thomas.durand+cv01@exemple.com | 06 22 14 73 90 | 38 rue des Tilleuls, 69009 Lyon
Profil
Dynamique, motivé et doté d'une grande capacité d'adaptation, je suis à la recherche d'un
poste de commercial en alternance où je pourrai mettre à profit mes compétences en
négociation et relation client.
Fort de mon expérience en tant que recruteur de donateurs et vendeur porte-à-porte, j'ai
développé une persévérance à toute épreuve, ainsi qu'une maîtrise des techniques de vente
directe et de gestion des objectifs.
Compétences, langues & centres d'intérêt
Compétences Langues Centres d'intérêt
Créatif Anglais Voyages
Assertivité Auteur & Compositeur amateur
Organisation d'évènement Sport : Basic fit et Basket
Animation et gestion de groupes
Bon relationnel
Polyvalent
Expérience professionnelle
Barman — Ginger Bar, Lyon (oct. 2024 - janv. 2025)
• Conception de cocktails et de snack
• Relation client et encaissement
Conseiller clientèle porte à porte — Bouygu